Load fashion MNIST, which is a drop-in replacement of MNIST. (70,000 grayscale images of 28 × 28 pixels each, with 10 classes)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# The class names are NOT included in the dataset, so
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Show some info
print("Tensorflow version: {:s}".format(tf.__version__))
print("Keras version: {:s}".format(keras.__version__))
print("X_train_full shape: {} and type: {}".format(X_train_full.shape, X_train_full.dtype))

# Scale pixel values down to the 0–1 range
# It's important that the training set and the testing set be preprocessed
X_train_full, X_test = X_train_full/255.0, X_test/255.0

# Create a validation set
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

Display the first 25 images from the training set and display the class name below each image

In [ ]:
plt.figure(figsize=(10,10))
for i in range(15):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train_full[i]])
plt.show(block=False)

In [ ]:
# Create a model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation="softmax")
])

# Show some info
model.summary()
# Note that Dense layers often have a lot of parameters. For example, the first hidden
# layer has 28×28× density = 784 × 128 connection weights, plus 128 bias terms
# This gives the model quite a lot of flexibility to fit the training
# data, but it also means that the model runs the risk of overfitting, especially when you
# do not have a lot of training data.

# Alternative way of constructing a model:
##model = keras.models.Sequential()
##model.add(keras.layers.Flatten(input_shape=[28, 28]))
##model.add(keras.layers.Dense(300, activation="relu"))
##model.add(keras.layers.Dense(100, activation="relu"))
##model.add(keras.layers.Dense(10, activation="softmax"))
##
##Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:
##
##model = tf.keras.models.Sequential([
##  tf.keras.layers.Flatten(input_shape=(28, 28)),
##  tf.keras.layers.Dense(128, activation='relu'),
##  tf.keras.layers.Dropout(0.2),
##  tf.keras.layers.Dense(10)
##])



Compile the model and shoose an optimizer.
We use the "sparse_categorical_cross entropy" loss because we have sparse labels

When using the SGD optimizer, it is important to tune the learning
rate. So, you will generally want to use optimizer=keras.optimiz
ers.SGD(lr=???) to set the learning rate, rather than opti
mizer="sgd", which defaults to lr=0.01.

In [ ]:
# Compile the model
model.compile(optimizer='adam', # 'sgd'
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Currently (dec 2020) it seems that cross-validation has not been implemented in Keras, so we would have to build something ourselves. No time for that right now

# Feed the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Show the learning curves

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
print(history.history.keys())
fig, ax = plt.subplots()
# Plot history: MAE
ax.plot(history.history['loss'], label='loss (training data)')
ax.plot(history.history['val_loss'], label='loss (validation data)')
ax.plot(history.history['accuracy'], label='accuracy (training data)')
ax.plot(history.history['val_accuracy'], label='accuracy (validation data)')
ax.set_title('Learning curves')
ax.set_ylabel('value')
ax.set_xlabel('No. epoch')
ax.grid(True)
ax.set_ylim(0,1)
ax.legend(loc="lower right")
plt.show()

Check performance on the test set

In [ ]:

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Show some predictions

In [ ]:
predictions = model.predict(X_test)
most_probable_predictions = np.argmax(predictions,axis=1)

# Plot confusion matrix
from sklearn.metrics import confusion_matrix

cm = np.round(confusion_matrix(y_test, most_probable_predictions, normalize='true'),1)
fig0, ax0 = plt.subplots(1,1)
ax0.imshow(cm, cmap=plt.cm.Blues)
ax0.set_xlabel("Predicted labels")
ax0.set_ylabel("True labels")
ax0.set_xticks(np.arange(len(class_names)))
ax0.set_yticks(np.arange(len(class_names)))
ax0.set_xticklabels(class_names, rotation = 45, ha="right")
ax0.set_yticklabels(class_names)
ax0.set_title('Confusion matrix ')
# ax0.set_colorbar()
plt.tight_layout()

import seaborn as sns
plt.figure()
ax4 = sns.heatmap(cm, cmap=plt.cm.Blues, annot=True, xticklabels=class_names, yticklabels=class_names)
plt.tight_layout()